In [1]:
import numpy
from context import vaeqst

# Quantum Device

## QuantumDevice

`QuantumDevice(state, circuit)` is a high-level interface to simulate the data acquisition protocol on a stabilizer state $\rho$ via a random Clifford circuit $U$.

**Parameters:**
* `state` - a `StabilizerState` representing the base state $\rho$.
* `circuit` - a `RandomCliffordCircuit` representing the unitary channel $U$ in the measurement protocol.

### Methods

#### measure

`QuantumDevice.measure(nsample)` provides a generator to serve $n_\text{sample}$ of classical snapshot states from the posterior POVM $\mathcal{E}_{\sigma|\rho}$.

* Forward protocol (physical):
    * Get a copy of $\rho$.
    * Evolve the state by the unitary circuit $\rho\to \rho'=U\rho U^\dagger$.
    * Measure in the computational basis $\rho'\to|b\rangle\langle b|$.
    * Collect the snapshot state $\hat{\sigma}=U^\dagger |b\rangle\langle b| U$.
    
* Backward protocol (computational):
    * Start from the all-up state $M_0=|\vec{0}\rangle\langle \vec{0}|=\prod_i\frac{1+Z_i}{2}$.
    * Transform backward through the unitary channel $M_0\to M= U^\dagger M_0 U=\prod_i\frac{1+Z'_i}{2}$ to the measurement basis.
    * Get a copy of $\rho$, measure observables $Z'_i$ , obtain measurement outcome $b_i$.
    * Collect the post measurement state $\hat{\sigma}=\prod_{i}\frac{1+(-)^{b_i}Z'_i}{2}=\prod_{i}\frac{1+g_i}{2}$ (by redefining $g_i=(-)^{b_i}Z'_i$).
    
Both protocols collect the classical snapshot $\hat{\sigma}$ correctly. The backward protocol is implement in the computation.

Example:

In [15]:
qdvc = vaeqst.QuantumDevice(vaeqst.GHZState(6), vaeqst.GlobalRCC(6))
qdvc

QuantumDevice(
    StabilizerState(
     +ZZIIII
     +IZZIII
     +IIZZII
     +IIIZZI
     +IIIIZZ
     +XXXXXX),
    |[0,1,2,3,4,5]|)

In [18]:
for snapshot in qdvc.measure(5):
    print(snapshot)
    print('expectation: ',qdvc.state.expect(snapshot))

StabilizerState(
 +ZXZXXZ
 -XIIXYI
 -ZIIYIY
 +YZYIXX
 +IYZYZY
 +YIXIZX)
expectation:  [0 0 0 0 0 0]
StabilizerState(
 +ZZIIII
 +ZZXIYI
 +YXYZZY
 -ZZIZYZ
 +IZZIXX
 +IZZYZZ)
expectation:  [1 0 0 0 0 0]
StabilizerState(
 +IYXYIY
 +YYZIZZ
 -YYXIYI
 -IXZZIX
 +ZIZIIX
 -YXXYXX)
expectation:  [0 0 0 0 0 1]
StabilizerState(
 +IYIIYI
 -YXZYZZ
 +IIZYYY
 -YXZIZZ
 -ZZXIXI
 -ZZZIZZ)
expectation:  [0 0 0 0 0 0]
StabilizerState(
 +IXZZXI
 +XZIZYZ
 +XIIYYX
 +ZZZIZI
 +XXIZIY
 +XXXXXX)
expectation:  [0 0 0 1 0 1]


### Fidelity Estimate

Reconstruction map for
* global Clifford measurement,
$$\tilde{\rho}=\mathcal{M}^{-1}[\sigma] = (2^N+1)\sigma-\mathbb{1}=\mathbb{E}_{\mathcal{E}_{\sigma|\rho}}(1+2^{-N})\prod_i(g_i+\mathbb{1})-\mathbb{1}$$
* local Clifford measurement,
$$\tilde{\rho}=\mathcal{M}^{-1}[\sigma] = \prod_i((2+1)\sigma_i-\mathbb{1}_i) = \mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\prod_i\frac{3g_i+\mathbb{1}}{2}.$$





$$F=\langle\rho\rangle=\mathbb{E}_{\mathcal{E}_{\sigma|\rho}}(1+2^{-N})\prod_i(1+\langle g_i\rangle)-1.$$

In [23]:
N = 50
nsample = 1000
qdvc = vaeqst.QuantumDevice(vaeqst.GHZState(N), vaeqst.GlobalRCC(N))
numpy.mean([(1+2**(-qdvc.N))*numpy.prod(1+qdvc.state.expect(snapshot))-1 for snapshot in qdvc.measure(nsample)])

1.0520000000000018